In [1]:
import numpy as np
import h5py
import keras
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import keras.backend as K
import tensorflow as tf
import yolo.config as cfg
from yolo.process import preprocess

Using TensorFlow backend.


In [6]:
from yolo.net.vgg16_net import yolo_vgg16_TFdim_model
from keras.applications.imagenet_utils import decode_predictions

In [3]:
model = yolo_vgg16_TFdim_model(is_top=True)

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv3_1 (Conv2D)             (None, 56, 56, 256)       295168    
__________

In [5]:
model.load_weights('models/pretrain/vgg16_tfdim_top_weights.h5')

In [7]:
imagePath = 'test/my_testset/car1.jpg'
image = preprocess(imagePath)
image = np.expand_dims(image, axis=0)
net_out = model.predict(image)
print(decode_predictions(net_out))

[[('n04285008', 'sports_car', 0.86457908), ('n04037443', 'racer', 0.079304308), ('n03100240', 'convertible', 0.039483458), ('n02974003', 'car_wheel', 0.0074502588), ('n02814533', 'beach_wagon', 0.0028537174)]]


## Weights load 확인완료

In [12]:
model.save_weights('models/pretrain/vgg16_tfdim_top_weights_named.h5')

In [10]:
model.save('models/pretrain/vgg16_tfdim_top_fullmodel_named.h5')

In [13]:
model_json = model.to_json()
with open("models/pretrain/vgg16_tfdim_top_fullmodel_named.json", "w") as json_file:
    json_file.write(model_json)
print("model json config write done..")

model json config write done..


## Depregate

In [14]:
pretrained_weights_path = 'models/pretrain/vgg16_weights.h5'
img_width, img_height, img_channel = 224, 224, 3

In [11]:
print(K.image_dim_ordering())
print(K.image_data_format())

tf
channels_last


In [19]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Conv2D(64, (3, 3), activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(64, (3, 3), activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(128,(3, 3), activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(128, (3, 3), activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(256, (3, 3), activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(256, (3, 3), activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(256, (3, 3), activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Conv2D(512, (3, 3), activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

f = h5py.File(pretrained_weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
    # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]

    if layer.__class__.__name__ in ['Conv1D', 'Conv2D', 'Conv3D', 'AtrousConvolution2D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))

    layer.set_weights(weights)
f.close()
print("VGG16 Model with No Top loaded...")
model_vgg.summary()

VGG16 Model with No Top loaded...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_40 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_41 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_42 (ZeroPaddi (None, 114, 114, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 1

In [22]:
type(model_vgg)

keras.models.Sequential

In [2]:
import numpy as np
import h5py
import keras
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import keras.backend as K
from keras.applications.vgg16 import VGG16

In [3]:
model = VGG16(weights='imagenet', include_top=False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [4]:
model.save_weights('models/pretrain/vgg16_tfdim_notop_weights.h5')

In [5]:
model.save('models/pretrain/vgg16_tfdim_notop_fullmodel.h5')

In [ ]:
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications.imagenet_utils import decode_predictions, preprocess_input

In [ ]:
from yolo.process import preprocess

In [ ]:
imagePath = 'test/my_testset/car1.jpg'
inputShape = (224,224,3)
# image = load_img(imagePath, target_size=inputShape)
# image = img_to_array(image)
# plt.imshow(image)
image = cv2.imread(imagePath).astype(np.float32)
print("1", image.shape)
image = preprocess(image)
print("2", image.shape)
image = np.expand_dims(image, axis=0)
print("3", image.shape)
# image = preprocess_input(image)
# print(image[0][0][0][0])
net_out = model.predict(image)
decoode_netout = decode_predictions(net_out)
print(decoode_netout)